# Setup PolyBase in SQL Server 2019
The following queries will setup PolyBase and create a connection to another SQL Server Instance.

Source: [GitHub SQL Tools](https://github.com/schenkfab/sqltools)

## Prerequisites
Make sure that PolyBase is installed and enabled.

In [ ]:
SELECT SERVERPROPERTY ('IsPolyBaseInstalled') AS IsPolyBaseInstalled; 
GO
EXEC sp_configure @configname = 'polybase enabled', @configvalue = 1;
RECONFIGURE WITH OVERRIDE ; 

## Create Test Database
Let's create a database which is going to hold the external tables.

In [ ]:
CREATE DATABASE [PolyBase]
GO

In [ ]:
USE [PolyBase]

## Create Master Key
The Master Key is going to be used to encrypt the secret of the Credentials

In [ ]:
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'ThisIsMyPassword:P@ssw0rd1'

## Create a Database Scoped Credential
The database scoped credential is being used to authenticate and authorize against the external data source. Remember, that the specified secret (password) is being encrypted by the MASTER KEY.

In [ ]:
CREATE DATABASE SCOPED CREDENTIAL CredentialForSQL
WITH IDENTITY = 'extLogin', SECRET = 'P@ssw0rd1';

## Create the External Data Source

In [ ]:
CREATE EXTERNAL DATA SOURCE SQL2017
    WITH ( LOCATION = 'sqlserver://localhost',
    PUSHDOWN = ON,
    CREDENTIAL = CredentialForSQL);

### PUSHDOWN
Pushdown pushed down `where` predicates to the external data source, if the predicate is one of the following:
* Binary comparision for numberic, date, and time values like: `=, !=, >, <`
* Arithmetic operators like `+` and `-`
* Logical operators like `AND` and `OR`
* Unary operators like `NOT`, `IS NULL`, `IS NOT NULL`

Possible pushdown for: `BETWEEN`, `NOT IN`, and `LIKE` depening on the query optimization phase.

## Create the External Table

In [ ]:
USE [PolyBase]
GO
CREATE EXTERNAL TABLE dbo.extDatabases(
    [name] SYSNAME,
    [database_id] INT
)
WITH (
    LOCATION='[master].[sys].[databases]',
    DATA_SOURCE=SQL2017
);

In [ ]:
SELECT *
  FROM dbo.extDatabases

In [ ]:
USE [PolyBase];
CREATE EXTERNAL TABLE [dbo].[DimAccount]
(
    [AccountKey] INT NOT NULL,
    [ParentAccountKey] INT,
    [AccountCodeAlternateKey] INT,
    [ParentAccountCodeAlternateKey] INT,
    [AccountDescription] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS,
    [AccountType] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS,
    [Operator] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS,
    [CustomMembers] NVARCHAR(300) COLLATE SQL_Latin1_General_CP1_CI_AS,
    [ValueType] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS,
    [CustomMemberOptions] NVARCHAR(200) COLLATE SQL_Latin1_General_CP1_CI_AS
)
WITH (LOCATION = N'[AdventureWorksDW2017].[dbo].[DimAccount]', DATA_SOURCE = [SQL2017]);

In [ ]:
SELECT * FROM dbo.DimAccount